In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid", color_codes=True)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense, Flatten, Conv2DTranspose,
    Reshape, BatchNormalization, Dropout, Input, ReLU, LeakyReLU
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

from PIL import Image

import warnings
warnings.filterwarnings('ignore')


In [5]:
import zipfile

with zipfile.ZipFile('/content/archive (3).zip', 'r') as zip_ref:
    zip_ref.extractall('anime_dataset')

In [ ]:
img_width, img_height = 256, 256
batchsize = 32

train = keras. utils.image_dataset_from_directory(
    directory='/content/anime_dataset',
    batch_size = batchsize,
    image_size = (img_width, img_height))

In [ ]:
iterator = iter(train)
images, labels = next(iterator)

fig, axes = plt.subplots(1, 4, figsize=(12, 6))  # 1 row, 4 columns
for i in range(4):
    axes[i].imshow(images[i].numpy().astype("uint8"))  # convert to uint8 for plotting
    axes[i].axis("off")                               # hide axes
    axes[i].set_title(str(labels[i].numpy()))         # show label as title

plt.tight_layout()
plt.show()


In [ ]:
# Path to the dataset folder
dataset_dir = '/content/anime_dataset'

# Create a data generator with rescaling and horizontal flipping
train_datagen = ImageDataGenerator(
    rescale=1./255,       # normalize pixel values to [0,1]
    horizontal_flip=True  # randomly flip images horizontally
)

# Load images from the directory as batches
train_generator = train_datagen.flow_from_directory(
    directory=dataset_dir,
    target_size=(64, 64),  # resize images
    batch_size=batchsize,
    class_mode=None         # no labels since this is unsupervised / DCGAN
)

# Inspect the first batch of images
images_batch = next(train_generator)
print("Batch shape:", images_batch.shape)


In [ ]:
# Kernel initializer
kernel_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
latent_dim = 300  # Dimension of random noise input

def build_generator(latent_dim=latent_dim):
    """
    Builds the Generator model for DCGAN.
    """
    model = keras.Sequential(name="Generator")

    # Dense layer to project random noise
    model.add(keras.layers.Dense(8 * 8 * 512, input_dim=latent_dim))
    model.add(keras.layers.ReLU())

    # Reshape to 3D tensor (8x8x512)
    model.add(keras.layers.Reshape((8, 8, 512)))

    # Upsampling layers
    model.add(keras.layers.Conv2DTranspose(
        256, (4, 4), strides=(2, 2),
        padding='same', kernel_initializer=kernel_init, activation='relu'
    ))
    model.add(keras.layers.Conv2DTranspose(
        128, (4, 4), strides=(2, 2),
        padding='same', kernel_initializer=kernel_init, activation='relu'
    ))
    model.add(keras.layers.Conv2DTranspose(
        64, (4, 4), strides=(2, 2),
        padding='same', kernel_initializer=kernel_init, activation='relu'
    ))

    # Output layer (RGB image)
    model.add(keras.layers.Conv2D(3, (4, 4), padding='same', activation='sigmoid'))

    return model

# Instantiate and summarize
generator = build_generator()
generator.summary()

# Visualize the model (optional)
keras.utils.plot_model(generator, show_shapes=True, show_layer_names=True)


In [ ]:
def build_discriminator(input_shape=(64, 64, 3)):
    """
    Builds the Discriminator model for DCGAN.
    """
    model = keras.Sequential(name="Discriminator")

    # Convolutional layers with LeakyReLU activations
    model.add(keras.layers.Conv2D(64, (3, 3), input_shape=input_shape))
    model.add(keras.layers.LeakyReLU(alpha=0.2))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(keras.layers.Conv2D(128, (3, 3)))
    model.add(keras.layers.LeakyReLU(alpha=0.2))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(keras.layers.Conv2D(256, (3, 3)))
    model.add(keras.layers.LeakyReLU(alpha=0.2))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

    # Flatten and fully connected layers
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(256))
    model.add(keras.layers.LeakyReLU(alpha=0.2))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    return model

# Instantiate and summarize
discriminator = build_discriminator()
discriminator.summary()

# Visualize discriminator architecture (optional)
keras.utils.plot_model(discriminator, show_shapes=True, show_layer_names=True)


In [15]:
class DCGAN(keras.Model):
    def __init__(self, generator, discriminator, latent_dim=latent_dim):
        super(DCGAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.latent_dim = latent_dim

        # Metrics to track losses
        self.g_loss_metric = keras.metrics.Mean(name='g_loss')
        self.d_loss_metric = keras.metrics.Mean(name='d_loss')

    @property
    def metrics(self):
        # List of metrics to reset at the start of each epoch
        return [self.g_loss_metric, self.d_loss_metric]

    def compile(self, g_optimizer, d_optimizer, loss_fn):
        super(DCGAN, self).compile()
        self.g_optimizer = g_optimizer
        self.d_optimizer = d_optimizer
        self.loss_fn = loss_fn

    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]

        # Generate random noise
        random_noise = tf.random.normal(shape=(batch_size, self.latent_dim))

        # -------------------------------
        # Train Discriminator
        # -------------------------------
        with tf.GradientTape() as tape:
            # Discriminator on real images
            pred_real = self.discriminator(real_images, training=True)
            real_labels = tf.ones((batch_size, 1))
            # Label smoothing
            real_labels += 0.05 * tf.random.uniform(tf.shape(real_labels))
            d_loss_real = self.loss_fn(real_labels, pred_real)

            # Discriminator on fake images
            fake_images = self.generator(random_noise, training=True)
            pred_fake = self.discriminator(fake_images, training=True)
            fake_labels = tf.zeros((batch_size, 1))
            d_loss_fake = self.loss_fn(fake_labels, pred_fake)

            # Total discriminator loss
            d_loss = (d_loss_real + d_loss_fake) / 2

        # Compute and apply gradients for discriminator
        d_gradients = tape.gradient(d_loss, self.discriminator.trainable_variables)
        self.d_optimizer.apply_gradients(zip(d_gradients, self.discriminator.trainable_variables))

        # -------------------------------
        # Train Generator
        # -------------------------------
        random_noise = tf.random.normal(shape=(batch_size, self.latent_dim))
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_noise, training=True)
            pred_fake = self.discriminator(fake_images, training=True)
            # Generator wants discriminator to classify fakes as real
            g_loss = self.loss_fn(tf.ones((batch_size, 1)), pred_fake)

        # Compute and apply gradients for generator
        g_gradients = tape.gradient(g_loss, self.generator.trainable_variables)
        self.g_optimizer.apply_gradients(zip(g_gradients, self.generator.trainable_variables))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)

        return {'d_loss': self.d_loss_metric.result(), 'g_loss': self.g_loss_metric.result()}


In [16]:
class DCGANMonitor(keras.callbacks.Callback):
    def __init__(self, num_imgs=25, latent_dim=latent_dim):
        super(DCGANMonitor, self).__init__()
        self.num_imgs = num_imgs
        self.latent_dim = latent_dim
        # Pre-generate random noise for monitoring
        self.fixed_noise = tf.random.normal([self.num_imgs, self.latent_dim])

    def on_epoch_end(self, epoch, logs=None):
        # Generate images from fixed noise
        generated_images = self.model.generator(self.fixed_noise, training=False)

        # Scale images from [0,1] to [0,255] for visualization
        generated_images = tf.clip_by_value(generated_images * 255, 0, 255)
        generated_images = generated_images.numpy().astype("uint8")

        # Optional: display a grid of generated images
        import matplotlib.pyplot as plt
        fig, axes = plt.subplots(5, 5, figsize=(10, 10))
        idx = 0
        for i in range(5):
            for j in range(5):
                axes[i, j].imshow(generated_images[idx])
                axes[i, j].axis('off')
                idx += 1
        plt.suptitle(f"Epoch {epoch+1}")
        plt.show()

    def on_train_end(self, logs=None):
        # Save the generator model after training
        self.model.generator.save('DCGAN_Generator.h5')


In [ ]:
epochs = 30
lr_g =0.0003
lr_d = 0.0001
beta = 0.5
latent_dim = 300

dcgan = DCGAN(generator=generator, discriminator=discriminator, latent_dim = latent_dim )
dcgan.compile(g_optimizer = Adam (learning_rate= lr_g, beta_1= beta), d_optimizer= Adam (learning_rate = lr_g , beta_1= beta), loss_fn = BinaryCrossentropy())

# Fit the model and save the history
history = dcgan.fit(train_generator, epochs=epochs, callbacks=[DCGANMonitor()])